In [242]:
# !pip install pyomo
# !pip install idaes-pse

In [243]:

import numpy as np
import pandas as pd
import time

from pyomo.environ import *

solvername = 'ipopt'


Caso não encontre o solver na pasta:
- Importar arquivos de extensões da biblioteca IDAES.

In [244]:
# search for solver executable in the PATH

import os

if os.path.exists('bin\\' + solvername + '.exe'):
    print(solvername + ' executable found')
    # add current folder to the PATH
    os.environ['PATH'] += ';.'

else:
    print(solvername + ' executable not found')

    # ------------------------- # 

    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'


ipopt executable found


In [245]:

sig = 0

UTEs = [('UTE1', 'UTE2', 'UTE3', 'UTE4'),
        (1000, 970, 700, 680),
        (16.19, 17.26, 16.6, 16.5),
        (0.00048, 0.00031, 0.002, 0.00211),
        (455, 455, 130, 130),
        (150, 150, 20, 20)]
UTEs = np.transpose(UTEs)
UTE = pd.DataFrame(UTEs, columns=['Nome','a','b','c','PGmax','PGmin'])



Dem = [('Hora1', 450, 45),
       ('Hora2', 530, 53),
       ('Hora3', 600, 60),
       ('Hora4', 540, 54),
       ('Hora5', 400, 40),
       ('Hora6', 280, 28),
       ('Hora7', 290, 29),
       ('Hora8', 500, 50)]
Dem = pd.DataFrame(Dem, columns=['Hora','Dem', 'Res'])

UTE['a'] = UTE['a'].astype('float')
UTE['b'] = UTE['b'].astype('float')
UTE['c'] = UTE['c'].astype('float')
UTE['PGmax'] = UTE['PGmax'].astype('float')
UTE['PGmin'] = UTE['PGmin'].astype('float')


In [246]:
# # mudar a demanda da hora 6, usando o iloc
# Dem.iloc[5,1] = 900
# Dem.iloc[5,2] = 90
# Dem

In [247]:
# Função para otimização do Unit Commitment

def Otimiza_UC(UTE,Dem,Sig,Print):
    #=====================================Modelagem do Problema============================================================
    modelo = ConcreteModel('Otimizacao Unit Commitment')

    # ----------------    Limites das Variáveis
    def limite_UTE(modelo,pg,ext):
        return (UTE[UTE['Nome'] == pg]['PGmin'].values[0], UTE[UTE['Nome'] == pg]['PGmax'].values[0])
    def limite_U(modelo,pg,ext):
        return (0, 1)
    def limite_X(modelo,pg,ext):
        return (0.0, 20.0)

    # ----------------    Declarar Variáveis
    modelo.Pg = Var(UTE['Nome'], Dem['Hora'], bounds=limite_UTE, domain=NonNegativeReals)
    modelo.U = Var(UTE['Nome'], Dem['Hora'], bounds=limite_U, domain=NonNegativeReals) # Função Decisão de Operação - FDO

    if Sig == 1:
        modelo.X = Var(UTE['Nome'], Dem['Hora'], bounds=limite_X, domain=NonNegativeReals) # Argumento da FDO  -  utilizado na aproximação sigmoidal


    # ----------------    Declarar Função Objetivo
    def objective_rule(modelo):
        fob = 0
        for h in Dem['Hora']:
            fob += sum((UTE[UTE['Nome'] == g]['a'].values[0] +
                        UTE[UTE['Nome'] == g]['b'].values[0]*modelo.Pg[g,h] +
                        UTE[UTE['Nome'] == g]['c'].values[0]*modelo.Pg[g,h]**2)*modelo.U[g,h] for g in UTE['Nome'])
        return fob


    modelo.objetivo = Objective(rule=objective_rule,sense=minimize)

    # ----------------     Declarar Restrições
    modelo.AtendDem = ConstraintList()

    for h in Dem['Hora']:
        modelo.AtendDem.add(sum(modelo.U[g,h]*modelo.Pg[g,h] for g in UTE['Nome']) == Dem[Dem['Hora'] == h]['Dem'].values[0])


    modelo.AtendDemRes = ConstraintList()

    for h in Dem['Hora']:
        modelo.AtendDemRes.add(sum(modelo.U[g,h]*UTE[UTE['Nome'] == g]['PGmax'].values[0] for g in UTE['Nome']) >=
                               Dem[Dem['Hora'] == h]['Dem'].values[0] + Dem[Dem['Hora'] == h]['Res'].values[0])

    # FDO Sigmoidal - definição

    if Sig == 1:
        modelo.FDO_Sigmoide = ConstraintList() # Restrição da FDO Sigmoidal
        alpha=10
        for h in Dem['Hora']:
            for g in UTE['Nome']:
                modelo.FDO_Sigmoide.add(modelo.U[g,h]  -  (exp(alpha * (modelo.X[g,h])) - 1) / (exp(alpha * (modelo.X[g,h])) + 1) == 0)


    # caso Sig == 0, o algoritmo decidirá o valor de U entre 0 e 1; logo, será feita uma aproximação linear

    # modelo.pprint()


    # ------------------------- #

    ### Executar a Otimização.
    # solver = SolverFactory('ipopt')
    # solver = SolverFactory('ipopt',executable='ipopt.exe')
    solver = SolverFactory('ipopt',executable='bin/ipopt')

    result = solver.solve(modelo, tee = (Print > 1))

    # Relatório dos resultados de otimização

    if Print >= 1:
        if Sig == 1:
            print(' - Função Decisão de Operação (FDO) Sigmoidal \n')
        else:
            print(' - Função Decisão de Operação (FDO) Linear \n')
        print('UNIT COMMITMENT')
        print('Status:', result.solver.status) # Status Final do Problema de Otimização
        print('Condição de Término:', result.solver.termination_condition)
        print('Resultado Função Objetivo: R$ ', value(modelo.objetivo),'\n')


    # ------------------------- #

    u_val = modelo.U.extract_values()

    df_u = pd.DataFrame(columns=UTE['Nome'], index=Dem['Hora'])
    for i in u_val:
        df_u[i[0]][i[1]] = u_val[i]


    # ordenar, para cada hora, as UTEs em ordem decrescente de U
    ordem_despacho = np.zeros((len(Dem['Hora']), len(UTE['Nome'])))
    for i in range(len(Dem['Hora'])):
        # guardar o número da UTE 
        ordem_despacho[i] = np.argsort(df_u.iloc[i])[::-1]

    ordem_despacho = ordem_despacho.astype(int)

    # print('Ordem de Despacho das UTEs:')
    # print(ordem_despacho)


    # ------------------------- #

    # Unit Commitment
    UC = np.zeros((len(Dem['Hora']), len(UTE['Nome']))).astype(int)

    for i in range(len(Dem['Hora'])):
        on = 0
        soma_ger_max = 0
        soma_ger_min = 0

        while soma_ger_max <= Dem['Dem'][i] + Dem['Res'][i]:
            soma_ger_max += UTE['PGmax'][ordem_despacho[i][on]]
            soma_ger_min += UTE['PGmin'][ordem_despacho[i][on]]

            if soma_ger_min <= Dem['Dem'][i]:
                UC[i][ordem_despacho[i][on]] = 1
            
            else:
                soma_ger_max -= UTE['PGmax'][ordem_despacho[i][on]]
                soma_ger_min -= UTE['PGmin'][ordem_despacho[i][on]]
                break
            
            on += 1


    return modelo, UC


In [248]:
# Função para otimização do Despacho

def Otimiza_Despacho(UTE,Dem,UC,Print):    

    #=====================================Modelagem do Problema============================================================
    modelo = ConcreteModel('Otimizacao do Despacho')
    
    # ----------------    Limites das Variáveis
    def limite_UTE(modelo,pg,ext):
        return (UTE[UTE['Nome'] == pg]['PGmin'].values[0], UTE[UTE['Nome'] == pg]['PGmax'].values[0])

    # ----------------    Declarar Variáveis
    modelo.Pg = Var(UTE['Nome'], Dem['Hora'], bounds=limite_UTE, domain=NonNegativeReals)

    # ----------------    Declarar Função Objetivo
    def calcular_fob(PG):
        fob = 0
        for h, Hora in enumerate(Dem['Hora']):
            for g in range(len(UTE['Nome'])):
                try:
                    fob += (UTE['a'][g] + UTE['b'][g]*PG[UTE['Nome'][g], Hora] + UTE['c'][g]*PG[UTE['Nome'][g], Hora]**2)*UC[h][g]
                except:
                    fob += (UTE['a'][g] + UTE['b'][g]*PG[UTE['Nome'][g]][Hora] + UTE['c'][g]*PG[UTE['Nome'][g]][Hora]**2)*UC[h][g]
        return fob

    def objective_rule(modelo):
        return calcular_fob(modelo.Pg)


    modelo.objetivo = Objective(rule=objective_rule,sense=minimize)

    # ----------------     Declarar Restrições
    modelo.AtendDem = ConstraintList()

    for h, Hora in enumerate(Dem['Hora']):
        atend_dem = 0
        for g, nome_g in enumerate(UTE['Nome']):
            atend_dem += modelo.Pg[nome_g, Hora]*UC[h][g]

        modelo.AtendDem.add(atend_dem == Dem['Dem'][h])

    
    # modelo.AtendDemRes = ConstraintList()

    # for h, Hora in enumerate(Dem['Hora']):
    #     atend_res = 0
    #     for g, nome_g in enumerate(UTE['Nome']):
    #         atend_res += modelo.Pg[nome_g, Hora]*UC[h][g]

    #     modelo.AtendDemRes.add(atend_res >= Dem['Dem'][h] + Dem['Res'][h])


    # modelo.UC_off = ConstraintList()

    # for h, Hora in enumerate(Dem['Hora']):
    #     uc_off = 0
    #     for g, nome_g in enumerate(UTE['Nome']):
    #         if UC[h][g] == 0:
    #             modelo.UC_off.add(modelo.Pg[nome_g, Hora] == 0)

        
    # modelo.pprint()


    # ------------------------- #

    ### Executar a Otimização.
    # solver = SolverFactory('ipopt')
    # solver = SolverFactory('ipopt',executable='ipopt.exe')
    solver = SolverFactory('ipopt',executable='bin/ipopt')

    result = solver.solve(modelo, tee = (Print > 1))


    # ------------------------- #

    pg_val = modelo.Pg.extract_values()

    df_pg = pd.DataFrame(columns=UTE['Nome'], index=Dem['Hora'])
    for i in pg_val:
        df_pg[i[0]][i[1]] = pg_val[i]
        if df_pg[i[0]][i[1]] == None:
            df_pg[i[0]][i[1]] = 0

    # Relatório dos resultados de otimização

    if Print >= 1:
        print('DESPACHO')
        print('Status:', result.solver.status) # Status Final do Problema de Otimização
        print('Condição de Término:', result.solver.termination_condition)
        print('Resultado Função Objetivo: R$ ', calcular_fob(df_pg), '\n')

    return modelo, df_pg


In [249]:
# Código Principal  -  Otimização Unit Commitment + Despacho

Print = 1  # 0: não imprimir nada;  1: imprimir resultados;  2: imprimir resultados e relatório do solver

t = time.time()

# Modelo com aproximação linear
modelo_lin, unit_commit_lin = Otimiza_UC(UTE, Dem, Sig=False, Print=Print)
# print(unit_commit_lin)
modelo_desp_lin, despacho_lin = Otimiza_Despacho(UTE, Dem, unit_commit_lin, Print=Print)

t_lin = time.time() - t
if Print >= 1:
    print('Tempo:', t_lin, 'segundos')
    print(1*'\n ' + 45*'-' + 2*'\n')


#  --------------------------- #


# Modelo com aproximação sigmoide
modelo_sig, unit_commit_sig = Otimiza_UC(UTE, Dem, Sig=True, Print=Print)
modelo_desp_sig, despacho_sig = Otimiza_Despacho(UTE, Dem, unit_commit_sig, Print=Print)

t_sig = time.time() - t - t_lin
if Print >= 1:
    print('Tempo:', t_sig, 'segundos')


 - Função Decisão de Operação (FDO) Linear 

UNIT COMMITMENT
Status: ok
Condição de Término: optimal
Resultado Função Objetivo: R$  67863.08835063892 

DESPACHO
Status: ok
Condição de Término: optimal
Resultado Função Objetivo: R$  72305.79512385899 

Tempo: 0.5961422920227051 segundos

 ---------------------------------------------


 - Função Decisão de Operação (FDO) Sigmoidal 

UNIT COMMITMENT
Status: ok
Condição de Término: optimal
Resultado Função Objetivo: R$  67863.0887873713 

DESPACHO
Status: ok
Condição de Término: optimal
Resultado Função Objetivo: R$  72305.79512385899 

Tempo: 0.1877884864807129 segundos


In [250]:
# rodar os modelos linear e sigmoide varias vezes, para comparar os tempos

n = 1000  # 1000

t_lin = np.zeros(n)
t_sig = np.zeros(n)

for i in range(n):
    t = time.time()
    modelo_lin, unit_commit_lin = Otimiza_UC(UTE, Dem, Sig=False, Print=0)
    modelo_desp_lin, despacho_lin = Otimiza_Despacho(UTE, Dem, unit_commit_lin, Print=0)
    t_lin[i] = time.time() - t

    t = time.time()
    modelo_sig, unit_commit_sig = Otimiza_UC(UTE, Dem, Sig=True, Print=0)
    modelo_desp_sig, despacho_sig = Otimiza_Despacho(UTE, Dem, unit_commit_sig, Print=0)
    t_sig[i] = time.time() - t

print('Tempo médio linear:', np.mean(t_lin), 'segundos')
print('Tempo médio sigmoide:', np.mean(t_sig), 'segundos')



Tempo médio linear: 0.16218308520317076 segundos
Tempo médio sigmoide: 0.17968673181533815 segundos


In [251]:
despacho_lin

Nome,UTE1,UTE2,UTE3,UTE4
Hora,,,,
Hora1,300.000001,150.0,0,0
Hora2,380.000001,150.0,0,0
Hora3,450.000001,150.0,0,0
Hora4,390.000001,150.0,0,0
Hora5,400.0,0,0,0
Hora6,280.0,0,0,0
Hora7,290.0,0,0,0
Hora8,350.000001,150.0,0,0


In [252]:
despacho_sig

Nome,UTE1,UTE2,UTE3,UTE4
Hora,,,,
Hora1,300.000001,150.0,0,0
Hora2,380.000001,150.0,0,0
Hora3,450.000001,150.0,0,0
Hora4,390.000001,150.0,0,0
Hora5,400.0,0,0,0
Hora6,280.0,0,0,0
Hora7,290.0,0,0,0
Hora8,350.000001,150.0,0,0
